In [1]:
pip install requests beautifulsoup4 PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [2]:
# The URL of the page with the PDF links
base_url = 'https://www.sanfranciscopolice.org/stay-safe/crime-data/crime-reports'

# Get the HTML content of the page
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the links that end with '.pdf'
pdf_links = [urljoin(base_url, a['href']) for a in soup.find_all('a', href=True) if a['href'].endswith('.pdf')]

# Download each PDF
for link in pdf_links:
    response = requests.get(link)
    filename = link.split('/')[-1]
    
    # Make sure the request was successful and the content-type is PDF before saving the file
    if response.status_code == 200 and 'application/pdf' in response.headers.get('Content-Type', ''):
        with open(filename, 'wb') as f:
            f.write(response.content)
            print(f'Downloaded {filename}')
    else:
        print(f'Failed to download {filename}')

Downloaded SFPDJanuary2023Compstat-20230210.pdf
Downloaded SFPDCompstatReport-February2023-20230313.pdf
Downloaded SFPDCrimeReport_March2023_20230424.pdf
Downloaded SFPDCrimeReport_April2023_20230512.pdf
Downloaded SFPDCrimeReport_May2023_20230621.pdf
Downloaded SFPDCrimeReport_June2023_20230807.pdf
Downloaded SFPDCrimeReport_July2023_20230831.pdf
Downloaded SFPDCrimeReport_Aug2023_20230920.pdf
Downloaded SFPDCompstatReportJan2022-20220301.pdf
Downloaded SFPDCompstatReportFeb2022-20220301.pdf
Downloaded SFPDCompstatReportMarch2022-20220419.pdf
Downloaded SFPDCrimeReport-April2022-20220606.pdf
Downloaded SFPDCrimeReportMay2022-20220616.pdf
Downloaded SFPDCompstatJune2022-20220801.pdf
Downloaded SFPDCompstatReport-July2022-20220901.pdf
Downloaded SFPDCrimeReportAugust2022-20220912.pdf
Downloaded SFPDCompstatReportSeptember2022-20221011.pdf
Downloaded SFPDCompstatOctober2022-20221116.pdf
Downloaded SFPDCompstat-Nov2022-20221216.pdf
Downloaded SFPDCompstat-Dec2022-20230111.pdf
Downloaded S

In [7]:
pip install pdfminer.six

  Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)
  Using cached cryptography-41.0.5-cp37-abi3-manylinux_2_28_x86_64.whl (4.4 MB)
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8
Note: you may need to restart the kernel to use updated packages.


In [8]:
from pdfminer.high_level import extract_text

def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

pdf_path = 'SFPD-COMPSTAT-YearEnd2015.pdf'

text = extract_text_from_pdf(pdf_path)
print(text)

COMPSTAT
CITY WIDE PROFILE
12/01/2015 to 12/31/2015

Chief Greg Suhr

CLASSIFICATION OF OFFENSE

ACTUAL OFFENSES

ACTUAL OFFENSES

2014

2015

DECEMBER

DECEMBER

1. CRIMINAL HOMICIDE TOTAL

2. FORCIBLE RAPE

A. Forcible Rape

B. Assault to Rape (Attempt)

TOTAL FOR FORCIBLE RAPE:

3. ROBBERY

A. Firearm

B. Knife or Cutting Instrument

C. Other Dangerous Weapon

D. Strongarm (no weapon)

TOTAL FOR ROBBERY:

4. AGGRAVATED ASSAULT

A. Firearm

B. Knife or Cutting Instrument

C. Other Dangerous Weapon

D. Hands, Fists, Feet, Etc.

TOTAL FOR AGGRAVATED ASSAULT:

5. BURGLARY

A. Forcible Entry

B. Unlawful Entry - No force

C. Attempted Forcible Entry

6. LARCENY THEFT

TOTAL FOR BURGLARY:

4

33

0

33

62

13

36

196

307

27

17

82

56

182

189

228

38

455

7

29

1

30

57

31

27

180

295

10

30

72

61

173

196

218

34

448

TOTAL FOR LARCENY THEFT:

3,676

2,613

7. MOTOR VEHICLE THEFT

A. Autos

B. Trucks and Buses

C. Other Vehicles

TOTAL FOR MOTOR VEHICLE THEFT:

8. ARS

In [9]:
import pandas as pd

data = text

lines = data.split("\n")

# Extract headers
headers = [line for line in lines if line.isupper() and len(line.split()) > 1]

# Extract data points
data_points = [line for line in lines if line.replace('.', '').isdigit() or line.replace('-', '').isdigit() or line.replace('%', '').isdigit()]

# Create a dictionary to map headers to their corresponding data points
mapped_data = {}
header_idx = 0
data_idx = 0
while header_idx < len(headers) and data_idx < len(data_points):
    header = headers[header_idx]
    if header not in mapped_data:
        mapped_data[header] = []
    
    mapped_data[header].append(data_points[data_idx])
    
    data_idx += 1
    if data_idx % 2 == 0:  # Since we have data for two years
        header_idx += 1

# Print the mapped data
for header, values in mapped_data.items():
    print(f"{header}: {values}")

CITY WIDE PROFILE: ['2014', '2015']
CLASSIFICATION OF OFFENSE: ['4', '33']
ACTUAL OFFENSES: ['0', '33', '62', '13', '22', '8', '0', '476']
1. CRIMINAL HOMICIDE TOTAL: ['36', '196']
2. FORCIBLE RAPE: ['307', '27']
TOTAL FOR FORCIBLE RAPE:: ['17', '82']
3. ROBBERY: ['56', '182']
TOTAL FOR ROBBERY:: ['189', '228']
4. AGGRAVATED ASSAULT: ['38', '455']
TOTAL FOR AGGRAVATED ASSAULT:: ['7', '29']
5. BURGLARY: ['1', '30']
6. LARCENY THEFT: ['57', '31']
TOTAL FOR BURGLARY:: ['27', '180']
TOTAL FOR LARCENY THEFT:: ['295', '10']
7. MOTOR VEHICLE THEFT: ['30', '72']
TOTAL FOR MOTOR VEHICLE THEFT:: ['61', '173']
8. ARSON .- TOTAL: ['196', '218']
9. HUMAN TRAFFICKING – SEX ACT .- TOTAL: ['34', '448']
10. HUMAN TRAFFICKING – INVOLUNTARY SERV.- TOTAL: ['441', '36']
GRAND TOTAL: ['44', '521']


In [10]:
import pandas as pd

# Classification of offenses as index
classification_of_offense = [
    "1. CRIMINAL HOMICIDE TOTAL",
    "2. FORCIBLE RAPE",
    "A. Forcible Rape",
    "B. Assault to Rape (Attempt)",
    "TOTAL FOR FORCIBLE RAPE:",
    "3. ROBBERY",
    "A. Firearm",
    "B. Knife or Cutting Instrument",
    "C. Other Dangerous Weapon",
    "D. Strongarm (no weapon)",
    "TOTAL FOR ROBBERY:",
    "4. AGGRAVATED ASSAULT",
    "A. Firearm",
    "B. Knife or Cutting Instrument",
    "C. Other Dangerous Weapon",
    "D. Hands, Fists, Feet, Etc.",
    "TOTAL FOR AGGRAVATED ASSAULT:",
    "5. BURGLARY",
    "A. Forcible Entry",
    "B. Unlawful Entry - No force",
    "C. Attempted Forcible Entry",
    "TOTAL FOR BURGLARY:",
    "6. LARCENY THEFT",
    "TOTAL FOR LARCENY THEFT:",
    "7. MOTOR VEHICLE THEFT",
    "A. Autos",
    "B. Trucks and Buses",
    "C. Other Vehicles",
    "TOTAL FOR MOTOR VEHICLE THEFT:",
    "8. ARSON .- TOTAL",
    "9. HUMAN TRAFFICKING – SEX ACT .- TOTAL",
    "10. HUMAN TRAFFICKING – INVOLUNTARY SERV.- TOTAL",
    "GRAND TOTAL"
]

# Columns of the table
columns = [
    "2014 DECEMBER ACTUAL OFFENSES",
    "2015 DECEMBER ACTUAL OFFENSES",
    "% Change",
    "2015 NOVEMBER ACTUAL OFFENSES",
    "2015 DECEMBER ACTUAL OFFENSES",
    "% Change",
    "2014 YTD ACTUAL OFFENSES",
    "2015 YTD ACTUAL OFFENSES",
    "% Change"
]

# Actual data extracted from the images
data = [
    [4, 7, 75, 3, 7, 133, 46, 52, 13],
    [' ',' ',' ',' ',' ',' ',' ',' ',' '],
    [33, 29, -12, 28, 29, 4, 364, 374, 3],
    [0, 1, "not cal", 0, 1, "not cal", 14, 17, 21],
    [33, 30, -9, 28, 30, 4, 378, 397, 5],
    [' ',' ',' ',' ',' ',' ',' ',' ',' '],
    [62, 57, -8, 38, 57, 50, 665, 618, -7],
    [13, 31, 138, 17, 31, 82, 274, 341, 24],
    [36, 27, -25, 29, 27, -7, 255, 242, 33],
    [13, 180, -86, 161, 180, 12, 2154, 3300, 86],
    [307, 295, -4, 245, 295, 20, 3348, 3621, 8],
    [' ',' ',' ',' ',' ',' ',' ',' ',' '],
    [27, 10, -63, 15, 10, -33, 284, 264, -7], 
    [17, 30, 76, 30, 30, 0, 370, 388, 5],
    [82, 72, -12, 95, 72, -24, 1040, 1149, 10],
    [56, 61, 9, 72, 61, -15, 1005, 907, -10],
    [182, 173, -5, 212, 173, -18, 2699, 2708, 0],
    [' ',' ',' ',' ',' ',' ',' ',' ',' '],
    [128, 196, 4, 150, 196, 31, 2080, 2109, 1],
    [189, 218, 44, 221, 218, -1, 3073, 2734, -11],
    [38, 34, -11, 18, 34, 89, 311, 343, 10],
    [455, 448, -2, 389, 448, 15, 5464, 5186, -5],
    [' ',' ',' ',' ',' ',' ',' ',' ',' '],
    [3676, 2613, -29, 2862, 2613, -9, 37466, 40971, 9],
    [' ',' ',' ',' ',' ',' ',' ',' ',' '],
    [441, 476, 8, 458, 476, 4, 5290, 5712, 8],
    [36, 51, 42, 38, 51, 34, 461, 437, -5],
    [44, 55, 25, 61, 55, -10, 592, 789, 33],
    [521, 582, 12, 557, 582, 4, 6343, 6938, 9],
    [22, 19, -14, 21, 19, -10, 241, 275, 14],
    [8, 4, 50, 1, 4, 300, 80, 67, -16],
    [0,0,0,0,0,0,0,0,0],
    [5208, 4171, -20, 4318, 4171, -3, 56065, 60209, 7],
]

# Create DataFrame
df = pd.DataFrame(data, index=classification_of_offense[:len(data)], columns=columns)

# Print DataFrame
df

,2014 DECEMBER ACTUAL OFFENSES,2015 DECEMBER ACTUAL OFFENSES,% Change,2015 NOVEMBER ACTUAL OFFENSES,2015 DECEMBER ACTUAL OFFENSES,% Change,2014 YTD ACTUAL OFFENSES,2015 YTD ACTUAL OFFENSES,% Change
1. CRIMINAL HOMICIDE TOTAL,4,7,75,3,7,133,46,52,13
2. FORCIBLE RAPE,,,,,,,,,
A. Forcible Rape,33,29,-12,28,29,4,364,374,3
B. Assault to Rape (Attempt),0,1,not cal,0,1,not cal,14,17,21
TOTAL FOR FORCIBLE RAPE:,33,30,-9,28,30,4,378,397,5
3. ROBBERY,,,,,,,,,
A. Firearm,62,57,-8,38,57,50,665,618,-7
B. Knife or Cutting Instrument,13,31,138,17,31,82,274,341,24
C. Other Dangerous Weapon,36,27,-25,29,27,-7,255,242,33
D. Strongarm (no weapon),13,180,-86,161,180,12,2154,3300,86


In [11]:
len(data)

33

In [13]:
from pdfminer.high_level import extract_text

def extract_text_from_pdf(pdf_path):
    # Extract text from PDF
    text = extract_text(pdf_path)
    # Split text into lines
    lines = text.split('\n')
    # Remove the first four lines
    clean_lines = lines[6:]
    # Join the remaining lines back into a single string
    clean_text = '\n'.join(clean_lines)
    return clean_text

pdf_path = 'SFPD-COMPSTAT-YearEnd2015.pdf'

# Extract and clean text from the PDF
text = extract_text_from_pdf(pdf_path)
print(text)

CLASSIFICATION OF OFFENSE

ACTUAL OFFENSES

ACTUAL OFFENSES

2014

2015

DECEMBER

DECEMBER

1. CRIMINAL HOMICIDE TOTAL

2. FORCIBLE RAPE

A. Forcible Rape

B. Assault to Rape (Attempt)

TOTAL FOR FORCIBLE RAPE:

3. ROBBERY

A. Firearm

B. Knife or Cutting Instrument

C. Other Dangerous Weapon

D. Strongarm (no weapon)

TOTAL FOR ROBBERY:

4. AGGRAVATED ASSAULT

A. Firearm

B. Knife or Cutting Instrument

C. Other Dangerous Weapon

D. Hands, Fists, Feet, Etc.

TOTAL FOR AGGRAVATED ASSAULT:

5. BURGLARY

A. Forcible Entry

B. Unlawful Entry - No force

C. Attempted Forcible Entry

6. LARCENY THEFT

TOTAL FOR BURGLARY:

4

33

0

33

62

13

36

196

307

27

17

82

56

182

189

228

38

455

7

29

1

30

57

31

27

180

295

10

30

72

61

173

196

218

34

448

TOTAL FOR LARCENY THEFT:

3,676

2,613

7. MOTOR VEHICLE THEFT

A. Autos

B. Trucks and Buses

C. Other Vehicles

TOTAL FOR MOTOR VEHICLE THEFT:

8. ARSON .- TOTAL

9. HUMAN TRAFFICKING – SEX ACT .- TOTAL

10. HUMAN TRAFFI